# Top-selling model

A model that only recommend top selling products to customers

- Identify top 12 selling items from 1st-7th day and 15th-21st day each month
- Check numbers items were actually purchased by returning users from 8th-14th and 22nd-28th day each month
- Use AOP@12 and MAP@12 to evaluate the results

Best selling items EDA on Kaggle: https://www.kaggle.com/negoto/best-selling-items-catalog-like-eda-of-articles

In [2]:
import pandas as pd
import numpy as np
import collaborative_filter as cf

In [3]:
%%time
transaction_raw = pd.read_csv("../data/transactions_train.csv")

Wall time: 25.2 s


In [4]:
eval_month = ['2018-10','2018-11','2018-12','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07',
              '2019-09','2019-10','2019-11','2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06',
              '2020-07','2020-08']
traintest_day = ['01','07','08','14','15','21','22','28']

In [5]:
eval_month[21]

'2020-08'

In [6]:
#AOP_results = []
MAP_results = []

for i in range(len(eval_month)): #i: 0-21
    for j in [0,4]: #j:0, 4
        
        mask_train = (transaction_raw.t_dat >= eval_month[i]+'-'+traintest_day[j]) & \
                     (transaction_raw.t_dat <= eval_month[i]+'-'+traintest_day[j+1])
        X_train = transaction_raw.loc[mask_train].reset_index(drop=True)[['customer_id','article_id']]
        mask_test = (transaction_raw.t_dat >= eval_month[i]+'-'+traintest_day[j+2]) & \
                    (transaction_raw.t_dat <= eval_month[i]+'-'+traintest_day[j+3])
        X_test = transaction_raw.loc[mask_test].reset_index(drop=True)[['customer_id','article_id']]
        
        #find out top 12 selling items in training data
        top12items = list(X_train.head(100000).article_id.value_counts()[0:12].index)
        
        #save top 12 selling items for users in a dict
        top12_items = {}
        for user in X_train.customer_id:
            top12_items[user] = top12items
        
        #calculate AOP@12 for each train-test group
        #AOP12_top = cf.AOP(top12_items, X_test)
        #AOP_results.append(AOP12_top)
        
        #calculate MAP@12 for each train-test group
        MAP12_top = cf.MAP(top12_items, X_test)
        MAP_results.append(MAP12_top)

100%|███████████████████████████████████| 72262/72262 [04:58<00:00, 242.09it/s]


Number of users: 13405


100%|███████████████████████████████████| 76681/76681 [06:36<00:00, 193.30it/s]


Number of users: 16497


100%|███████████████████████████████████| 70960/70960 [03:31<00:00, 336.23it/s]


...

Number of users: 15254


100%|███████████████████████████████████| 84327/84327 [05:26<00:00, 258.67it/s]


Number of users: 15385


100%|███████████████████████████████████| 65673/65673 [04:25<00:00, 247.79it/s]

Number of users: 12733


In [7]:
MAP_results

[0.003706189889516992,
 0.0036500039011859277,
 0.004228553247026634,
 0.003982120916660209,
 0.003742289898411289,
 0.002934736693920944,
 0.009112088645093734,
 0.0059122713837938675,
 0.004957282720536082,
 0.0032645174690629137,
 0.0033463580357755178,
 0.001673560849002102,
 0.0034629062308753297,
 0.0017223648371323531,
 0.002257396037260413,
 0.002570213012212404,
 0.0018787974176229152,
 0.002069212079801238,
 0.0033659140890861423,
 0.0038606570579611684,
 0.004427217805418718,
 0.003914958878423329,
 0.0017535171966071845,
 0.004244672041610195,
 0.0040628891110973285,
 0.0055945002302185895,
 0.0009893937897049933,
 0.0028832436966839442,
 0.002564746230505089,
 0.0033560230714048526,
 0.002874857419588757,
 0.0033939079842479967,
 0.00457556546185097,
 0.005482219686089836,
 0.004470963912065659,
 0.0053007463389727226,
 0.002440746105883477,
 0.003017364288094463,
 0.0021104560227518075,
 0.0021949185909159656,
 0.0020705445102192086,
 0.0030071404559243735,
 0.00216880138

In [12]:
timeline = []
for month in eval_month:
    month_mid = month + '-15'
    month_late = month + '-22'
    timeline.append(month_mid)
    timeline.append(month_late)

timeline

44

In [14]:
result_df = pd.DataFrame({'timeline':timeline,
                          'MAP':MAP_results})
result_df

,timeline,MAP
0,2018-10-15,0.003706
1,2018-10-22,0.003650
2,2018-11-15,0.004229
3,2018-11-22,0.003982
4,2018-12-15,0.003742
5,2018-12-22,0.002935
6,2019-01-15,0.009112
7,2019-01-22,0.005912
8,2019-02-15,0.004957
9,2019-02-22,0.003265


In [15]:
result_df.to_csv("../data/top_sell_MAP12.csv")

In [3]:
#extract data between 2019-5-1 and 2019-6-1
mask = (transaction_raw.t_dat >= '2019-05-01') & (transaction_raw.t_dat <= '2019-06-01')
transaction_2019_May = transaction_raw.loc[mask].reset_index(drop=True)
transaction_2019_May

,t_dat,customer_id,article_id,price,sales_channel_id
0,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,524061003,0.050831,2
1,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,735404001,0.050831,2
2,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,700370004,0.016932,2
3,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,618800001,0.033881,2
4,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,731407001,0.016932,2
...,...,...,...,...,...
1622933,2019-06-01,fffdaab7d9d79ceee4ffd2ab98bb68411438ea90e46a67...,738881002,0.016932,2
1622934,2019-06-01,fffdaab7d9d79ceee4ffd2ab98bb68411438ea90e46a67...,220094010,0.025407,2
1622935,2019-06-01,ffff4c4e8b57b633c1ddf8fbd53db16b962cf831baf9ed...,487800001,0.011847,2
1622936,2019-06-01,ffff4c4e8b57b633c1ddf8fbd53db16b962cf831baf9ed...,754013001,0.022017,2


In [8]:
len(MAP_results)

44

In [3]:
#save 2019 May data to csv
#transaction_2019_May.to_csv("../data/transactions_2019_May.csv", index=False)

In [4]:
#read 2019 May transaction data
transaction_2019_May = pd.read_csv("../data/transactions_2019_May.csv")
transaction_2019_May

,t_dat,customer_id,article_id,price,sales_channel_id
0,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,524061003,0.050831,2
1,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,735404001,0.050831,2
2,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,700370004,0.016932,2
3,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,618800001,0.033881,2
4,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,731407001,0.016932,2
...,...,...,...,...,...
1622933,2019-06-01,fffdaab7d9d79ceee4ffd2ab98bb68411438ea90e46a67...,738881002,0.016932,2
1622934,2019-06-01,fffdaab7d9d79ceee4ffd2ab98bb68411438ea90e46a67...,220094010,0.025407,2
1622935,2019-06-01,ffff4c4e8b57b633c1ddf8fbd53db16b962cf831baf9ed...,487800001,0.011847,2
1622936,2019-06-01,ffff4c4e8b57b633c1ddf8fbd53db16b962cf831baf9ed...,754013001,0.022017,2


In [4]:
#extract data between 2019-5-1 and 2019-5-10
mask = (transaction_2019_May.t_dat >= '2019-05-01') & (transaction_2019_May.t_dat <= '2019-05-10')
X_train = transaction_2019_May.loc[mask].reset_index(drop=True)
X_train

,t_dat,customer_id,article_id,price,sales_channel_id
0,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,524061003,0.050831,2
1,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,735404001,0.050831,2
2,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,700370004,0.016932,2
3,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,618800001,0.033881,2
4,2019-05-01,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,731407001,0.016932,2
...,...,...,...,...,...
499329,2019-05-10,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,609837003,0.027102,2
499330,2019-05-10,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,638842003,0.049136,2
499331,2019-05-10,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,510675001,0.027102,2
499332,2019-05-10,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,771116006,0.067780,2


In [5]:
X_train = X_train[['customer_id','article_id']]
X_train

,customer_id,article_id
0,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,524061003
1,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,735404001
2,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,700370004
3,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,618800001
4,00019d6c20e0fbb551af18c57149af4707ec016bb0decd...,731407001
...,...,...
499329,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,609837003
499330,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,638842003
499331,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,510675001
499332,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,771116006


In [6]:
#check number of customers and items
print("Number of unique customer id:",X_train.head(100000).customer_id.nunique())
print("Number of unique article id:", X_train.head(100000).article_id.nunique())

Number of unique customer id: 24031
Number of unique article id: 15187


In [ ]:
%%time
recommend_items = cf.all_user_collaborative_filter(X_train.head(100000), similar_user_number = 100, max_recommend_items = 12)

In [11]:
mask = (transaction_2019_May.t_dat >= '2019-05-11') & (transaction_2019_May.t_dat <= '2019-05-17')
X_test = transaction_2019_May.loc[mask].reset_index(drop=True)
X_test = X_test[['customer_id','article_id']]
X_test

,customer_id,article_id
0,000416152a81973bebab23142bd5dd6ebd85c8c5e2d63e...,695544005
1,000416152a81973bebab23142bd5dd6ebd85c8c5e2d63e...,695632011
2,0006d3ff0caf0cb4d4e0615ee5cb7d268622364d483335...,748269002
3,00130faf36f2571cf7e08451b317545004a8a85327661f...,742191001
4,00130faf36f2571cf7e08451b317545004a8a85327661f...,347200001
...,...,...
331833,fff62d03f7f60e61a987a1b2c27a553dbae8a632df1322...,745475010
331834,fff62d03f7f60e61a987a1b2c27a553dbae8a632df1322...,708855002
331835,fff62d03f7f60e61a987a1b2c27a553dbae8a632df1322...,696128001
331836,fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...,700079006


In [30]:
MAP12_cf = cf.MAP(recommend_items, X_test)
print("User-based collaborative filter MAP@12:", MAP12_cf)

100%|███████████████████████████████████████████████████████████████████████████| 24031/24031 [01:18<00:00, 307.11it/s]

Number of users: 4332
User-based collaborative filter MAP@12: 0.0037136724997412925


In [31]:
##Identify top 12 selling items for training data
top12items = list(X_train.head(100000).article_id.value_counts()[0:12].index)

#save top 12 selling items for users in a dict
top12_items = {}
for user in recommend_items:
    top12_items[user] = top12items

In [33]:
MAP12_top = cf.MAP(top12_items, X_test)
print("Top 12 selling products MAP@12:", MAP12_top)

100%|███████████████████████████████████████████████████████████████████████████| 24031/24031 [01:21<00:00, 295.16it/s]

Number of users: 4332
Top 12 selling products MAP@12: 0.0020729474647988057


In [38]:
# % increase in MAP@12
print("MAP@12 increases by", round(100*(MAP12_cf-MAP12_top)/MAP12_top), "%")

MAP@12 increases by 79 %
